### Q-learning algorithm for Grid World problem

In [180]:
from enum import Enum
import random
import matplotlib.pyplot as plt
import numpy as np

#### Enviroment
Model represeting enviroment of Grid World, where agent need to find the best path to reward.

In [181]:
class Action(Enum):
    UP = 'UP'
    DOWN = "DOWN"
    LEFT = "LEFT"
    RIGHT = "RIGHT"

In [182]:
class GridWorldEnv:
    CONST_PENALTY = -0.1   

    states_actions = {
        ((0,0), Action.DOWN) : [(1., ((0,1), CONST_PENALTY))],
        ((0,0), Action.RIGHT) : [(1., ((1,0), CONST_PENALTY))],
        ((1,0), Action.LEFT) : [(1., ((0,0), CONST_PENALTY))],
        ((1,0), Action.RIGHT) : [(1., ((2,0), CONST_PENALTY))],
        ((2,0), Action.LEFT) : [(1., ((1,0), CONST_PENALTY))],
        ((2,0), Action.RIGHT) : [(.1, ((3,0), 1.)), (.9, ((1,0), CONST_PENALTY))],
        ((2,0), Action.DOWN) : [(1., ((2,1), CONST_PENALTY))],
        ((0,1), Action.DOWN) : [(1., ((0,2), CONST_PENALTY))],
        ((0,1), Action.UP) : [(1., ((0,0), CONST_PENALTY))],
        ((2,1), Action.UP) : [(.7, ((2,0), CONST_PENALTY)), (.3, ((3,1), -1.))],
        ((2,1), Action.DOWN) : [(1., ((2,1), CONST_PENALTY))],
        ((2,1), Action.RIGHT) : [(1., ((3,1), -1.))],
        ((0,2), Action.UP) : [(1., ((0,1), CONST_PENALTY))],
        ((0,2), Action.RIGHT) : [(1., ((1,2), CONST_PENALTY))],
        ((1,2), Action.RIGHT) : [(1., ((2,2), CONST_PENALTY))],
        ((1,2), Action.LEFT) : [(1., ((0,2), CONST_PENALTY))],
        ((2,2), Action.RIGHT) : [(1., ((3,2), CONST_PENALTY))],
        ((2,2), Action.LEFT) : [(1., ((1,2), CONST_PENALTY))],
        ((2,2), Action.UP) : [(1., ((2,1), CONST_PENALTY))],
        ((3,2), Action.LEFT) : [(1., ((2,2), CONST_PENALTY))],
        ((3,2), Action.UP) : [(1., ((3,1), -1.))],
    }

    actions = {
        (0,0) : {Action.DOWN, Action.RIGHT},
        (1,0) : {Action.LEFT, Action.RIGHT},
        (2,0) : {Action.LEFT, Action.RIGHT, Action.DOWN},
        (3,0) : {},
        (0,1) : {Action.UP, Action.DOWN},
        (2,1) : {Action.UP, Action.DOWN, Action.RIGHT},
        (3,1) : {},
        (0,2) : {Action.UP, Action.RIGHT},
        (1,2) : {Action.RIGHT, Action.LEFT},
        (2,2) : {Action.RIGHT, Action.LEFT, Action.UP},
        (3,2) : {Action.LEFT, Action.UP}
    }

    def env_return(self, state, action):
        probabilities, return_value = zip(*self.states_actions[(state, action)])
        return random.choices(return_value, weights=probabilities, k=1)[0]

In [183]:
class ActionValue:    
    def __init__(self, env):    
        self.Q = {}
        for (state, action) in env.states_actions:
            self.Q[(state, action)] = (random.random() * 2) - 1
    
    def get_max_action(self, state):
        max_a = None
        max_value = float('-inf')
        for (s,a) in self.Q:
            if s == state and self.Q[(s,a)] > max_value:
                max_a = a
                max_value = self.Q[(s,a)]
        return max_a, max_value
            

In [184]:
class Policy:
    def __init__(self, env: GridWorldEnv, Q: ActionValue, epsilon):
        self.env = env
        self.Q = Q
        self.epsilon = epsilon
    
    def get_action(self, state):
        if self.epsilon > random.random():
            return random.choice(list(self.env.actions[state]))
        else:
            return self.Q.get_max_action(state)[0]

In [192]:
env = GridWorldEnv()
Q = ActionValue(env)
policy = Policy(env=env, Q=Q, epsilon=.1)
alpha = .01
gamma = .1   

In [193]:
for _ in range(1000):
    current_state = (0,2)
    while env.actions[current_state] != {}:
        action = policy.get_action(current_state)
        next_state, reward = env.env_return(current_state, action)
        cur_Q = Q.Q[current_state, action]
        Q.Q[current_state, action] = cur_Q + alpha*(reward + gamma*Q.get_max_action(next_state)[1] - cur_Q)
        current_state = next_state

In [195]:
display(Q.Q)

{((0, 0), <Action.DOWN: 'DOWN'>): -0.11107780587452647,
 ((0, 0), <Action.RIGHT: 'RIGHT'>): -0.1111110960222626,
 ((1, 0), <Action.LEFT: 'LEFT'>): -0.11110773409695733,
 ((1, 0), <Action.RIGHT: 'RIGHT'>): -0.11111111067428155,
 ((2, 0), <Action.LEFT: 'LEFT'>): -0.11111095025184597,
 ((2, 0), <Action.RIGHT: 'RIGHT'>): nan,
 ((2, 0), <Action.DOWN: 'DOWN'>): -0.30056889730333153,
 ((0, 1), <Action.DOWN: 'DOWN'>): -0.11110417722399714,
 ((0, 1), <Action.UP: 'UP'>): -0.1111093137229863,
 ((2, 1), <Action.UP: 'UP'>): nan,
 ((2, 1), <Action.DOWN: 'DOWN'>): -0.11111111111111036,
 ((2, 1), <Action.RIGHT: 'RIGHT'>): nan,
 ((0, 2), <Action.UP: 'UP'>): -0.11109769736066707,
 ((0, 2), <Action.RIGHT: 'RIGHT'>): -0.11109774850173088,
 ((1, 2), <Action.RIGHT: 'RIGHT'>): -0.11111100366677246,
 ((1, 2), <Action.LEFT: 'LEFT'>): -0.12106963510458993,
 ((2, 2), <Action.RIGHT: 'RIGHT'>): -0.11111111111111188,
 ((2, 2), <Action.LEFT: 'LEFT'>): -0.11321595386420673,
 ((2, 2), <Action.UP: 'UP'>): -0.1182972133